In [1]:
from datasets import load_dataset, DatasetDict, Dataset
import numpy as np
imdb = load_dataset("imdb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [2]:
N = 2000
rand_indx = np.random.randint(0,24999,size=N)

x_train = imdb['train'][rand_indx]['text']
y_train = imdb['train'][rand_indx]['label']
x_test = imdb['test'][rand_indx]['text']
y_test = imdb['test'][rand_indx]['label']

In [3]:
dataset = DatasetDict({'train':Dataset.from_dict({'text':x_train,'label':y_train}),'validation':Dataset.from_dict({'text':x_test,'label':y_test})})

In [4]:
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch

In [5]:
print(torch.cuda.get_device_name(0))

Tesla T4


In [6]:
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}
model_checkpoint = 'distilbert-base-uncased'

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = 2, id2label = id2label,label2id = label2id)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space = True)
#add prefix space not needed for distilbert but needed for roberta so it is a good practice to add it to tokenizer
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token':'[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
#jezeli nie ma automatycznego dodawania pad keidy text za krotki to trzeba o to zadbac
def tokenize_function(example):
    text = example['text']
    tokenizer.truncation_side = 'left' #from start
    tokenizer_inputs = tokenizer(text, return_tensors='np', max_length=512, truncation=True)
    return tokenizer_inputs


tokenized_dataset = dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:

accuracy = evaluate.load('accuracy')
def compute_metrics(p):
    score, label = p
    predictions = np.argmax(score,1)

    return {'accuracy':accuracy.compute(predictions=predictions, references=label)}

In [9]:
text_list=["i dont care","i am fine with that","great idea","i hate this","skibidi","worst movie ever","not a fan","worst"]

for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt")
    logits = model(inputs).logits
    predictions = torch.argmax(logits)
    print( text +" "+ id2label[predictions.tolist()])

i dont care Negative
i am fine with that Negative
great idea Negative
i hate this Negative
skibidi Positive
worst movie ever Negative
not a fan Negative
worst Negative


In [10]:
peft_config = LoraConfig(task_type = "SEQ_CLS",r=4,lora_alpha = 8, lora_dropout =0.02, target_modules =['q_lin'])

model = get_peft_model(model,peft_config)
model.print_trainable_parameters()
# define training arguments



trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [11]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 3


In [12]:

training_args = TrainingArguments(
    output_dir=model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"   # disables wandb or other logging
)


In [ ]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

/tmp/ipython-input-769452018.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.417300,0.364685,{'accuracy': 0.892}
2,0.281100,0.507025,{'accuracy': 0.8715}


In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)
    logits = model(inputs).logits
    predictions = torch.argmax(logits, dim=1)
    print(text + " - " + id2label[predictions.item()])

